# Gcloud Setup

To keep sessions alive for long periods of time:

`gcloud compute ssh mlrl-vm --ssh-flag="-ServerAliveInterval=30"`

# Lecture 1 - K Nearest Neighbors

## Load Data

In [45]:
# pkl to dict
def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        dict_ = pickle.load(fo, encoding='bytes')
    return dict_

cifar_pwd = '../data/cifar-10-batches-py/data_batch_1'
cifar_data = unpickle(cifar_pwd)


## Split into Train / Test Sets

In [86]:
import numpy as np 

# split to test and train 

images = cifar_data[b'data']
labels = cifar_data[b'labels']

def split_data_set(x,y,p=0.9):
    n = x.shape[0]
    y = np.array(y)
    rand_indices = np.random.choice(range(n),n,replace=False)
    i = int(p*n)
    x_train,y_train = x[rand_indices[:i]],y[rand_indices[:i]]
    x_test,y_test = x[rand_indices[i:]],y[rand_indices[i:]]
    return x_train,y_train,x_test,y_test

x_train,y_train,x_test,y_test = split_data_set(images,labels)
x_train,y_train,x_val,y_val =  split_data_set(x_train,y_train)


## k Nearest Neighbors Classifier

In [87]:
import numpy as np
from scipy import stats

class NearestNeighbor:
    
    def __init__(self):
        pass
    
    def train(self,x,y):
        self.x_train = x
        self.y_train = y
    
    def predict(self,x,k=10,n=0):
        n = x.shape[0] if not n else n
        
        y_pred = np.zeros(n,dtype=np.dtype(self.y_train[0]))
        
        for i in range(n):
            distances = self.distance(self.x_train,x[i,:],'L2')
            min_indices = np.argsort(distances)[:k]
            k_nearest = list(self.y_train[min_indices])
            mode = max(set(k_nearest), key=k_nearest.count)
            y_pred[i] = mode
        return y_pred
        
    def distance(self,v1,v2,measure='L1'):
        if measure == 'L1':
            return np.sum(np.abs(v1-v2),1)
        elif measure == 'L2':
            return np.sqrt(np.sum((v1-v2)**2,1))
        else:
            raise ValueError('Please select L1 or L2 measure')
            
    
    def accuracy(self,pred,y):
        assert len(pred) == len(y), 'pred and y must have equal lengths'
        accuracy = sum([i==j for i,j in zip(pred,y)])/len(y)
        print('Accuracy:',str(round(100*accuracy,2))+'%','N:',len(y))
        return accuracy

nn = NearestNeighbor()
nn.train(x_train,y_train)
pred = nn.predict(x_test,k=6,n=15)
accuracy = nn.accuracy(pred,y_test[:15])

Accuracy: 20.0% N: 15


0.2

## Tuning K as Hyperparameter

In [90]:
for k in [1, 3, 5, 10, 20, 50, 100]:
    pred = nn.predict(x_test,k=k,n=100)
    print('K =',k)
    accuracy = nn.accuracy(pred,y_test[:100])
    print('='*20)

    

K = 1
Accuracy: 17.0% N: 100
K = 3
Accuracy: 18.0% N: 100
K = 5
Accuracy: 15.0% N: 100
K = 10
Accuracy: 22.0% N: 100
K = 20
Accuracy: 24.0% N: 100
K = 50
Accuracy: 23.0% N: 100
K = 100
Accuracy: 20.0% N: 100


# N-Fold Validation

When you don't have enough data, you can perform cross-validation. As an example let N=5. So you shuffle your data, and then split it into 5 equal sized folds. Select one as validation set and run your algorithm. Repeat process 5 times, each time using a different fold as validation and return the final performance average.

## High Level (interesting) Thoughts on ML 
https://homes.cs.washington.edu/~pedrod/papers/cacm12.pdf

In [1]:
import numpy as np

x = np.zeros((5,3))
x[1,:] = np.ones(3)
x

array([[0., 0., 0.],
       [1., 1., 1.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]])

In [3]:

train = np.ones((50,100))
test = np.ones((13,100))

dists = np.zeros((13,50))
train = train.T


In [9]:
x = np.sum(train,1).reshape(-1,1)
y = np.sum(test,1).reshape(-1,1)

dists[1,:]

(50,)

# Lecture 2 Linear Models

## Multiclass SVM

Multiclass SVMs have the following loss per data tuple $(x_i,y_i)$:

$$\sum_i L_i = \sum_{j \neq y_i} \max (0,s_j - s_{y_i} + \Delta) + \lambda \sum_{jk} W_{jk}^2$$

where

* $j$ is the number of distinct classes
* $s_j = w_j^T x_i$ is the linear prediction
* $w_j$ is the $j$th row of the weight matrix $W$
* $W$ is the weight matrix with shape $(N_f,N_c + 1)$. $N_c = \sum_j 1$ is the total number of classes, and the $+1$ is an additional term for the bias (i.e. bias trick). $N_f$ is the number of features.
* $\Delta$ is a margin of error term. It controls how far away a wrong prediction $s_{j \neq y_i}$ needs to be from the correct one in order to get penalized. Usually, we set it to $1$ if there's a weight regularization term.
* $\lambda$ is a regularization term

The final loss is the sum of the $i$ losses $L=\sum_i L_i$.

## Binary SVM

A binary SVM has the loss

$$L = \sum_i \max(0,1 - y_i w^T x_i) + \lambda \sum_j w_j^2$$

* Note that here there is no weight matrix $W$, only one weight vector $w$ with dimention $N_f$.

### Equivalence Between Binary and Multiclass SVMs when $N_C=2$ 
The binary SVM is a special case of the Multiclass SVM. You can derive equivalence in the following way:

1. Parametrize $j=-1,1$ as the two binary classes.
2. Then the weight matrix $W = [ w_{1},w_{-1}]^T$.
3. Let $w_1 = - w_{-1}$ so now we only have one weight vector $w \equiv w_1$.
4. Substitute these values into the multiclass SVM for both $y_i=1$ and $y_i=-1$ cases.
5. Noting that $\Delta$ and $\lambda$ are scaling factors that are inversely related, you will arrive at the binary loss written above.

## Softmax Model

This is multiclass logistic regression. The loss for element $i$ is defined as

$$L_i = -\sum_j  \frac{e^{s_{y_i} }}{e^{s_j}}$$

Equivalently, taking the log we get $L \rightarrow \log L$:

$$L_i = - s_{y_i}  + \log \sum_j {e^{s_j}}$$

Since the softmax function is $\text{softmax}(z_i) = \sum_j \exp(z_i) / \exp(z_j)$ this is called the Softmax Loss. Finally the explicit form of the loss $L_i$ is 

$$L_i = - w_{y_i}^T x_i + C  + \log \sum_j {e^{w_j^T x_i + C}}$$

where $C$ is a constant for numerical stability. We want to set $C$ to avoid division by zero. In practice, we set $C = - \max (w_j^T x_i)$.

## Computing the Gradient

The lecture is slightly incorrect in its notation. The score is $s_j = w_j^T x$ where $w_j$ is a column vector from the matrix $W$ which has shape $(D,C)$ - where $D$ is dimension and $C$ is classes. So $w_j$ has shape $(D,)$.

The gradient can be written analytically. For class $k$ it is:

$$\frac{\partial L_i}{\partial w_k} = - \mathbb 1 (x_i,k=y_i) + \frac{x_i e^{w_k^T x_i +C}}{\sum_j e^{w_j^T x_i+C}}$$

where $\mathbb 1$ is the indicator function (1 if condition is true, 0 otherwise).

## Information Theory Perspective

The information entropy between two distributions $p$ and $q$ is 

$$ H(p,q) = \sum_x p(x) \log q(x)$$

The softmax loss tries to minimize the information entropy between the true value $p(y_i)=1$ and the predicted value $p(j)$:

$$H(p(y_i),p(j)) = H(1,f(s_j)) = \sum_j \log \text{softmax}(s_j)$$

Since the information entropy can also be rewritten as 

$$H(p,q) = H(p) + D_{KL} (p\Vert q)$$

and $H(p(y_i)) = 0$, the loss is also trying to minimize the $KL$ divergence (distance) between the real distribution $p$ and the predicted on $q$. Cool stuff!

## Probabilistic Interpretation 

Also worth noting the probabilistic interpretation that 

$$P(y_i | x_i;W)  = \text{softmax} (s_{y_i})$$

By minimizing $L$ we are in fact maximizing the likelihood $P(y_i|x_i;W)$ so a softmax classifier can be interpreted as a likelihood maximization algorithm.

Finally, the minimization of the regularization term $R(w)$ can be interpreted as the assumption that weights $w$ come from a Gaussian prior distribution centered around mean $0$ and are therefore penalized when they take on values far away from the mean. Also cool stuff!

## Normalization trick

For numerical stability, rewrite 

$$\text {softmax }(z_i) = \sum_j \exp(z_i)/\exp(z_j)  =\sum_j C \exp(z_i)/ C \exp(z_j) = \sum_j \exp(z_i+\log C)/\exp(z_j+\log C)$$

Choose $C = \exp(- \max_k s_k)$ for the loss, so that the exponent in $\exp(s_j + \log C)$ is always $\leq 0$, meaning that it's impossible to divide by zero in the denominator.


# Lecture 3: Stochastic Gradient Descent

Before gradient descent one could try

1. Set weights randomly, train, and take best set - gets to 15% on CIFAR
2. Perturb W by a small delta W and take the new W+delta W that increases performance most - 20% on CIFAR
3. Compute the numerical gradient - performs well but takes a long time
4. Do stochastic gradient descent

http://cs231n.github.io/optimization-1/

## Numerical Approximation

Practical consideration is to use the centered difference derivative 

$$\partial f(x) = [f(x+h)−f(x−h)]/2h$$

This is good for reference, but the numerical gradient scales as $O(N_p)$ where $N_p$ is the number of paramters. For a 1 layers CIFAR network $N_d \approx 30K $, which is already large and it only gets worse for more complicated networks.



# Lecture 4: Backprop Intuition

Very cool lecture. Presents backprop as a circuit with gates as arithmetic operations on real-valued inputs. http://cs231n.github.io/optimization-2/

Some interesting examples of gates & consequences of backprop:

1. Additive gate $x+y$ - distributes gradients equally to all inputs (b.c. grad f(x) = 1 if f(x) = x).
2. Multipication gate $x*y$ - distributes gradients inversely to inputs (e.g. if $x$ is big $y$ is small, the gradients will be distributed more to $y$ and less to $x$, because grad x f(x,y) = y if f(x,y)=xy). As a result this can lead to unintuitive consequences. Small data values will be changed more that large ones. Important to normalize.
3. Max gate $\max (x,y)$ - will distribute gradient to the larger of the values.

# Lecture 5: NNs Part 1

About the biological analogy 
https://neurophysics.ucsd.edu/courses/physics_171/annurev.neuro.28.061604.135703.pdf
https://www.sciencedirect.com/science/article/abs/pii/S0959438814000130

## Remark

1. Regularization can be interpreted as biological forgetting, since weights (synaptic strengths) are driven to zero.

## Activation Functions

1. Sigmoid has following undesirable properties
    1. saturates gradients - the gradient of a sigmoid is zero near $\sigma(x) = 0,1$, meaning that during backprop the updates may be negligible and the network stops learning.
    2. Sigmoids are not zero centered. Since their output is always greater or equal to zero it means that gradients will be updated in one direction (all pos or all neg)
2. Tanh 
    - same as sigmoid but without undesireable property (B) since its output ranges from -1 to 1.
3. ReLU 
    - somewhat solves problem (A) because the gradients above zero is always 1, never saturating to zero.
    - easy to compute both forward and backward pass
    - can irreversibly die - the 0 threshold part can kill relu neurons
4. Leaky ReLU
    - solves dying ReLU problem 
5. Maxout
    - maxout neuron outputs maximum if linear inputs https://arxiv.org/pdf/1302.4389.pdf

    
More reading
https://arxiv.org/abs/1502.01852

## Universal Function Approximators

http://www.dartmouth.edu/~gvc/Cybenko_MCSS.pdf

http://neuralnetworksanddeeplearning.com/chap4.html

## Depth of Network

https://arxiv.org/abs/1312.6184

https://arxiv.org/abs/1412.6550

http://www.deeplearningbook.org/contents/mlp.html

## Number of Neurons

The more neurons that larger the network capacity the more complex functions the network can approximate. It's almost always necessary to regularize. 

One interesting observation is that smaller networks are less prone to overfitting because they have less parameters. Why do we not use shallow networks with limited numbers of neurons then? 

Interestingly, small networks have smoother / less complex loss surfaces. You'd think that would be a good thing, but it turns out it's easy to converge to suboptimal local minima. Deep networks, on the other hand, have much noisier loss surfaces with many more local minima, but because those are localized, it's easier to get out of them.

https://arxiv.org/abs/1412.0233

# Lecture 6: Setting Up Neural Networks


## Data Preprocessing 

### Basic Preprocessing

1. Mean substraction
    1. subtracting mean of every featyres
    2. subtracting mean of all pixels (for images)
2. Normalization
    1. Divide by std, so that std=1
    2. compress min, max data values to (-1,1)

Warning - the mean should be computed only across the training data, and then also subtracted from the test and val sets.

### More Advanced Preprocessing

1. PCA

```
import numpy as np

k = 3
x -= x.mean(0)
N = x.shape[0]
cov = x.T @ x / N # covariance between ith and jth feature
U,S,V = np.linalg.svd(cov) # singular value decomposition
                           # U - cols of U are eigenvectors
                           # S - singular values
x_projected = x @ U[:,:k]  # reduces from D dimension to k
return x_projected
```

2. Whitening - divide the reduced data by its eigenvalues

```
x_projected /= np.sqrt(S[:k]+1e-5) # S vals are the eigen vals squared
```

Warning - whitening normalizes the eigenspace, which means that noisy dimensions (where most of the data is uncorrelated with the outcome) become exaggerated.

## Weight Initialization

Never initialize weights to zero - then all gradients are the same and nothing is learned.

Initialize weights randomly. Heuristically, initialize each weight such that

`w = np.random.randn(n) / sqrt(n)`

where `n` is the number of inputs. The reason is that the variance is proportional to `n` meaning that the std is proportional to `sqrt(n)`. Dividing by the std normalizes the weights and scales them to a similar distribution.

http://proceedings.mlr.press/v9/glorot10a/glorot10a.pdf

https://arxiv.org/abs/1502.01852

Current recommendation is to use ReLU and to do `w = np.random.randn(n) * sqrt(2.0/n)` for both weights and biases.

### Batch Normalization

Forces activations to take on unit Gaussian distributions at the beginning of training. Usually, you put a batch norm layer immediately after dense or conv layer.

https://arxiv.org/abs/1502.03167

## Regularization

1. L2 reg - penalize $\frac 1 2 \lambda W^2$
2. L1 reg - penalize $\lambda |W|$
3. Elastic net - L1+L2
4. Max norm constraint - clip weight vectors with some upper bound e.g. $||w||_2 < C$ for some constant $C$. In practice, you do one step of gradient descent and then enforce the constraint by clipping the weights.
5. Dropout - randomly drop neurons during forward pass and backprop. Can be interpreted as selecting sub-networks and then the master network becomes an ensemble of a bunch of subnetworks.
    - Note 1: for predicting, nothing is dropped, but outputs are scaled by dropout probability $p$.
    - Note 2: it's best to perform scaling by $p$ during train time to keep test time performance fast.
    - drop out paper: http://www.cs.toronto.edu/~rsalakhu/papers/srivastava14a.pdf
    - expository paper: http://papers.nips.cc/paper/4882-dropout-training-as-adaptive-regularization.pdf
    - drop connect: sets weights randomly to zero in forward pass https://cs.nyu.edu/~wanli/dropc/
    
## Noise Theme

Dropout is part of a general theme that adding noise during training can be good.

## Loss Functions

### Multiclass Classification

SVM Loss 

$$L_i = \sum_{j\neq y_i} \max(0,s_j(x_i) - s_{y_i}(x_i) +1)$$

Softmax loss

$$L_i = - \log \Big (\frac{ \exp (s_{y_i} (x_i))}{  \sum_j  \exp (s_{j} (x_i)) }\Big )$$

Problem: when $N_c >>1$ the number of classes is large. You can then use a hierarchical softmax, which represented classes as a tree.

https://arxiv.org/pdf/1310.4546.pdf

### Binary Classification (or Multiple Attribute)

You can use the binary SVM and binary logistic loss.

### Regression

Use the mean squared error.

Note: Regression loss is much less stable than softmax. Always best to see if you can turn regression problem into a classification (e.g. binning).

One example where the L2 MSE loss fails is with outliers. Outliers can generate large L2 loss values, whereas they don't have as large of an effect on the Softmax loss.

### Structured Prediction

N/A

# Lecture 6: Neural Networks Part 3 Learning and Evaluation

## *important lecture

This is a highly practical boss lecture

http://cs231n.github.io/neural-networks-3/

## Tricks to building good models

### Gradient Check

Use numerical approximations to check the gradient. Specifically use the centered derivative formula

$$ \partial_x f(x+h) = \frac{f(x+h)-f(x-h)}{2h}$$

The formula has an error of $O(h^2)$ whereas the normal derivative formula has an error pf $O(h)$.

### Relative Error Comparison

Since $h$ is small but the gradient can also be small, it's important to scale the error appropriately.

$$ \frac{|f_a' - f_n'|}{\max(|f_a'|,|f_n'|)}$$

### Floating Point Issues

#### Use Double Precision

Again, since the gradients and $h$ are both small, it's important to use double floating point precision to avoid numerical errors.

#### Stay in Active Floating Point Range

Read here https://docs.oracle.com/cd/E19957-01/806-3568/ncg_goldberg.html

### Be Mindful of Kinks in the Objective Function

Many useful functions in NNs have kinks - a discontinuous term that looks like $\max(x,y)$ (e.g. SVM loss and ReLU). The larger the network and dataset being evaluated the more errors you may have due to kinks in the numerical approximation. The best way to debug with kink functions is to use a small dataset and small network, so that the odds of encountering a derivative near a kink are much smaller.

### Use only a few datapoints

Errors compound with data, so use only a few data points for debugging.

### Be careful with step size $h$

https://en.wikipedia.org/wiki/Numerical_differentiation

### Check gradients after a few epochs

You want to check gradients once the loss function is decreasing. Otherwise, you'll be checking the gradients of randomized weights, which could be equal to random numerical noise.

### Make sure regularization doesn't kill data signal

Either set the regularization parameter to zero or remove the regularization term for the first run, and then add it in.

### Turn off non-deterministic effects such as Dropout or Augmentation

You want to make sure your gradients work across the network.

### Check few dimensions per parameter

## Sanity Checks

### Check the loss

Make sure the loss is returning an initial value that makes sense.
- e.g. CIFAR & Softmax the initial loss without training should be around $-\log (0.1) \approx 2.3$
- e.g. CIFAR & SVM the initial loss without training should be $9$.

### Increase Regularization

This should increase the loss

### Overfit on a tiny subset of data

Take a small sample (say 20 data points) and train the model on it. The loss should go to zero, and the model should overfit.


## Useful Heuristics for Training

### Check loss shape

### Check validation / training accuracy for overfitting

### Measure ratio of update magnitudes to values

The intuition here is that the values should be larger than the update magnitudes (since update magnitudes change the values incrementally), but this ratio should be roughly constant around `1e-3`.

```
value_norm = np.linalg.norm(W.ravel())
update = lr*dW
update_norm = np.linalg.norm(update)
print(update_norm / value_norm) # should be around 1e-3
```

### Activation / Gradient Distributions Per Layer

You want activations / gradients to be distributed randomly initially. 

### Plot images of features in Conv Nets

Plotting the features, if they are smooth and extract visible shapes then that's a good sign you're on the right path. If the features are noisy, that might be a symptom of a bug.

# Gradient Update Schemes

## Vanilla GD

This is simply `x += - lr * dx`

## Momentum

First we introduce a new variable `v`, which is initialized to zero and has a scaling paramter $\mu =0.9$.

`v += mu * v - lr * dx`

And then the update rule is 

`x+=v`

Momentum is a misnomer, the value `v` acts like friction, dampening the gradient update velocity. For directions with a consistent gradient `v` accumulates velocity. The parameter is ofen annealed from `mu=0.5 -> .99` over training.

## Nesterov Momentum

Similar to momentum, but instead of computing on `x` you need to compute on the vector `x_ahead = mu*v +x`. Relabeling `x_ahead -> x` we get

```
v_prev = v
v = mu*v - lr * dx
x += mu*v_prev + (1-mu)*v # Polyak averaging
```

https://arxiv.org/pdf/1212.0901v2.pdf

See section 7.2 of 

http://www.cs.utoronto.ca/~ilya/pubs/ilya_sutskever_phd_thesis.pdf



## Anneal learning rate

1. Step decay - decrease $\alpha$ every $t$ steps
2. Exponential decay $\alpha = \alpha_0 e^{-kt}$
3. Inverse decay $\alpha = \alpha_0 / (1+kt)$

In practice, step decay works well.

## Newtons Method

Simple update rule, no learning rate

$$x \leftarrow x - [Hf(x)]^{-1} \nabla f(x)$$

$H$ is the hessian (square matrix of second order derivatives `(n_params,n_params)`. This form is clean but not practical for large data sets since `n_params` can be on the order of millions of large deep networks.

https://arxiv.org/abs/1311.2115

https://ai.google/research/pubs/pub40565

## Adagrad

Simple method to normalize gradients (keep them from getting too big or small for any parameter)

```
cache = dx**2
update = - lr*dx / np.sqrt(cache + epsilon)
x += update
```

## RMSProp

http://www.cs.toronto.edu/~tijmen/csc321/slides/lecture_slides_lec6.pdf

```
cache = decay*cache + (1-decay)*dx**2 # moving average
update = - lr*dx / np.sqrt(cache + epsilon)
x += update
```

## Adam

RMSProp + momentum. Simple form:

```
m = decay1*m + (1-decay1)*dx # moving average of gradient vector
v = decay2*v + (1-decay2)*dx**2 # moving average of gradient magnitude
update = - lr*m / np.sqrt(v + epsilon)
x+=update
```

Full form invloves bias terms. `m,v` both start as zero and are biased to it, so we need a bias correction mechanism.

```
m = decay1*m + (1-decay1)*dx # moving average of gradient vector
mt = m / (1+decay1**t) # bias term
v = decay2*v + (1-decay2)*dx**2 # moving average of gradient magnitude
vt = v / (1+decay2**t) # bias term
update = - lr*mt / np.sqrt(vt + epsilon)
x+=update
```

## Unit Tests 

https://arxiv.org/abs/1312.6055

## Hyperparameter Optimization

Common parameters

1. Learning rate
2. Learning decay rates
3. Regularization strength

Start with coarse and move to fine hyperparameter tuning.

### Worker / Master paradigm:
- worker samples hyperparameters randomly and starts executng
- there can be many workers per deployment
- master monitors worker performance and kills workers if they're not performing well

### One validation fold > cross-validation

When there is a lot of data it's better to keep one validation fold

### Hyperparameter Range

Most hyperparameters are iterated in log scale. Hyperparameters have mostly multiplicative effects, this is why it's better to use a log scale.

`learning_rate = 10 ** uniform(-6, 1)`

### Random grid search

Random grid seach is usually easier to implement and works better than iterative grid search.

http://www.jmlr.org/papers/volume13/bergstra12a/bergstra12a.pdf

### Bayesian Hyperparemeter search

Exploration/Exploitation schemes for parameter search

## Evaluation

Things to check during evaluation

1. Same model different initialization
2. Use cross validation to find best parameters
3. Running average of parameters during training

### Ensembles

https://www.youtube.com/watch?v=EK61htlw8hY

# Lecture 7: Putting it together: Minimal Neural Network Case Study 

All work is in assignment 1


## Feature Preprocessing

Preprocessing using a HOG + HSV color binning increases performance.

HOG tutorial:
https://lilianweng.github.io/lil-log/2017/10/29/object-recognition-for-dummies-part-1.html#histogram-of-oriented-gradients-hog